# Create and Validate CCE

This notebook demonstrates how to submit candidate content for validation and display the results.


## Setup


In [ ]:
# Install SDK
!pip install -e /content/dorc-clients/sdk/python


In [ ]:
import os
import pandas as pd
from dorc_client import DorcClient

# Configure environment
os.environ["DORC_BASE_URL"] = "https://your-engine-url.run.app"
os.environ["DORC_TENANT_SLUG"] = "my-tenant"

# Create client
client = DorcClient()


## Step 1: Define Candidate Content

Edit the `candidate_text` variable below with your content to validate.


In [ ]:
# Your candidate content (markdown)
candidate_text = """# My Document

This is the content I want to validate against the existing canon.

## Section 1

Some content here that might contradict existing canon.
"""

# Optional: Candidate identifier
candidate_id = "my-candidate-001"

print(f"Candidate ID: {candidate_id}")
print(f"Content length: {len(candidate_text)} characters")


## Step 2: Validate Content


In [ ]:
# Submit for validation
print("Submitting candidate for validation...")
response = client.validate(
    content=candidate_text,
    candidate_id=candidate_id,
)

print(f"\n✅ Validation complete!")
print(f"Run ID: {response.run_id}")
print(f"Pipeline Status: {response.pipeline_status}")


## Step 3: Display Summary


In [ ]:
# Display summary counts
summary = response.content_summary
total = summary.pass_ + summary.fail + summary.warn + summary.error

print("Validation Summary:")
print(f"  ✅ PASS:  {summary.pass_:3d}/{total}")
print(f"  ⚠️  WARN:  {summary.warn:3d}/{total} (non-blocking)")
print(f"  ❌ FAIL:  {summary.fail:3d}/{total} (blocking)")
if summary.error > 0:
    print(f"  🔴 ERROR: {summary.error:3d}/{total} (technical)")

print(f"\n{'='*60}")
if summary.fail > 0:
    print("❌ VALIDATION FAILED - Blocking contradictions found")
elif summary.warn > 0:
    print("⚠️  VALIDATION PASSED WITH WARNINGS - Review recommended")
else:
    print("✅ VALIDATION PASSED - No issues found")
print(f"{'='*60}")


## Step 4: Display Chunk Results


In [ ]:
# Create DataFrame for chunk results
chunk_data = []
for chunk in response.chunks:
    chunk_data.append({
        "Index": chunk.index,
        "Status": chunk.status,
        "Message": chunk.message,
        "Findings": chunk.finding_count,
        "Model": chunk.model_used or "N/A",
        "Chunk ID": chunk.chunk_id[:20] + "..."
    })

df = pd.DataFrame(chunk_data)
display(df)


## Step 5: View Detailed Results (Optional)


In [ ]:
# Display detailed results for chunks with findings
for chunk in response.chunks:
    if chunk.finding_count > 0 or chunk.status in ["FAIL", "WARN"]:
        status_icon = {
            "PASS": "✅",
            "WARN": "⚠️ ",
            "FAIL": "❌",
            "ERROR": "🔴"
        }.get(chunk.status, "❓")
        
        print(f"\n{status_icon} Chunk {chunk.index} ({chunk.status})")
        print(f"   Message: {chunk.message}")
        print(f"   Findings: {chunk.finding_count}")
        
        if chunk.evidence:
            print(f"   Evidence:")
            for ev in chunk.evidence:
                if ev.source:
                    print(f"     - Source: {ev.source}")
                if ev.note:
                    print(f"       Note: {ev.note}")
                if ev.excerpt:
                    print(f"       Excerpt: {ev.excerpt[:100]}...")
